In [1]:
import sys
import warnings
import pandas as pd
import numpy as np
import os
import cx_Oracle as O
import datetime as datetime
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime, date, timedelta
import settings
import errhandler as eh
import eHSQL
import re
import os
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.auth import HTTPBasicAuth
import json
from lxml import html
import string
import bs4
import selenium

global r, rnumjson


def getData():
    
    global dfpivot, df_test, dfsol, dfnum, dfden, recnum, payload, r, rnumjson, dfLE, session, auth, agent, cookie, cookie2
    
    agent = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.52'}
    
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    
    url1 = "https://essence.syndromicsurveillance.org/"

    session = requests.session()
    
    #session = AsyncHTMLSession()
    
    url1result = session.get(url1, headers = agent)
    
    print(f'\n{url1result.headers}\n', f'\n{url1result.request.headers}\n')
    
    print(session.cookies.get_dict())
    print(url1result.headers['Set-Cookie'])
    cookie = str(url1result.headers['Set-Cookie'])
    cookie = ' ' + cookie.replace(' Path=/; Secure; HttpOnly','').replace(';', '')
    print(cookie, '\n')
    
    url2 = "https://essence.syndromicsurveillance.org/nssp_essence"
    url2result = session.get(url2, headers=agent)
    
    print(f'\n{url2result.headers}\n', f'\n{url2result.request.headers}\n')
    
    print(session.cookies.get_dict())
    print(url2result.request.headers['Cookie'])
    cookie2 = str(url2result.request.headers['Cookie']).split()
    cookie2 = cookie2[0] + ' '
    print(cookie2, '\n')
    
    url3 = "https://essence.syndromicsurveillance.org/nssp_essence/servlet/Login"
    url3result = session.get(url3, headers=agent)
    
    print(f'\n{url3result.headers}\n', f'\n{url3result.request.headers}\n')
    
    tree = html.fromstring(url3result.text)
    authenticity_token = list(set(tree.xpath("//input[@name='csrfPreventionSalt']/@value")))[0]
    
    print(f'{authenticity_token}\n')
    
    payload = {"userId": "nservice01",
                   "password": "Testingacc!1",
                   "csrfPreventionSalt": f"{authenticity_token}"}
    
    loginpostresult = session.post(url3,
                                    data = payload,
                                    headers = agent,
                                    allow_redirects = True)
    
    print(f'\n{loginpostresult.headers}\n', f'\n{loginpostresult.request.headers}\n')
    
    url4 = "https://essence.syndromicsurveillance.org/nssp_essence/j_spring_security_check"
    url4result = session.get(url4, headers=agent)
    
    print(f'\n{url4result.headers}\n', f'\n{url4result.request.headers}\n')

    dfLE = pd.read_csv(r'C:\Users\gmoye\LegalEntities.csv')
    dfLE.rename(columns={'LE_Name':'erFacility'}, inplace = True)

    url5 = "https://essence.syndromicsurveillance.org/nssp_essence/servlet/TableBuilderPortletRedirectServlet?percentParam=noPercent&patientClass=e&endDate=4Aug2018&medicalGroupingSystem=essencesyndromes&userId=647&geographySystem=state&ccddCategory=ili%20ccdd%20v1&geography=nd&datasource=va_er&timeResolution=weekly&erFacility=6326&erFacility=6330&erFacility=6360&erFacility=6365&erFacility=6358&erFacility=6320&erFacility=6354&erFacility=6347&erFacility=6369&aqtTarget=TableBuilder&detector=nodetectordetector&startDate=29Jul2018&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayZeroCountRows=true&displayZeroCountRows=true&fieldIDs=timeResolution&fieldIDs=ageCDCILI&fieldIDs=erFacility&fieldIDs=geographystate&fieldLabels=Week&fieldLabels=CDC%20ILI%20Reporting%20Age%20Group&fieldLabels=Facility&fieldLabels=State&graphWidth=372&portletId=285297&dateconfig=16"
    url5result = session.get(url5, headers=agent)
    
    print(f'\n{url5result.headers}\n', f'\n{url5result.request.headers}\n')
    
    url6 = "https://essence.syndromicsurveillance.org/nssp_essence/servlet/TableBuilder?percentParam=noPercent&patientClass=e&medicalGroupingSystem=essencesyndromes&userId=647&geographySystem=state&ccddCategory=ili%20ccdd%20v1&geography=nd&datasource=va_er&timeResolution=weekly&erFacility=6326&erFacility=6330&erFacility=6360&erFacility=6365&erFacility=6358&erFacility=6320&erFacility=6354&erFacility=6347&erFacility=6369&aqtTarget=TableBuilder&detector=nodetectordetector&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayZeroCountRows=true&displayZeroCountRows=true&fieldIDs=timeResolution&fieldIDs=ageCDCILI&fieldIDs=erFacility&fieldIDs=geographystate&fieldLabels=Week&fieldLabels=CDC%20ILI%20Reporting%20Age%20Group&fieldLabels=Facility&fieldLabels=State&graphWidth=372&portletId=285297&dateconfig=16&startDate=06Nov22&endDate=12Nov22"
    url6result = session.get(url6, headers=agent)
    
    print(f'\n{url6result.headers}\n', f'\n{url6result.request.headers}\n')
    
def transformation():
    global denvarlist, numvarlist, dfdenweek, dfnumweek, startdate, enddate, periodlist, weeknum
    
    enddate = datetime.today().date()
    
    startdate = enddate - timedelta(days=14)
    
    enddate = datetime.strptime(str(enddate), '%Y-%m-%d').strftime('%d%b%y')
    
    startdate = datetime.strptime(str(startdate), '%Y-%m-%d').strftime('%d%b%y')
    #startdate = datetime.strptime('2022-01-17', '%Y-%m-%d').strftime('%d%b%y')
    
    urlnumjson = f'https://essence.syndromicsurveillance.org/nssp_essence/api/tableBuilder?percentParam=noPercent&patientClass=e&medicalGroupingSystem=essencesyndromes&userId=647&geographySystem=state&ccddCategory=ili ccdd v1&geography=nd&datasource=va_er&timeResolution=weekly&erFacility=6326&erFacility=6330&erFacility=6360&erFacility=6365&erFacility=6358&erFacility=6320&erFacility=6354&erFacility=6347&erFacility=6369&aqtTarget=TableBuilder&detector=nodetectordetector&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayZeroCountRows=true&displayZeroCountRows=true&fieldIDs=timeResolution&fieldIDs=ageCDCILI&fieldIDs=erFacility&fieldIDs=geographystate&fieldLabels=Week&fieldLabels=CDC ILI Reporting Age Group&fieldLabels=Facility&fieldLabels=State&graphWidth=372&portletId=285297&dateconfig=16&startDate={startdate}&endDate={enddate}&rowFields=erFacility&rowFields=ageCDCILI&rowFields=timeResolution&columnField=geographystate'
    
    with open(r'C:\Users\gmoye\headerconfignum.json') as g:
        headersnum = json.load(g)
    
    print(cookie + headersnum['cookie'] + cookie2, '\n')
    
    headersnum['cookie'] = (cookie2 + headersnum['cookie'] + cookie) 
    print(headersnum, '\n')
    print('\n')
    
    retry_strategy = Retry(total = 0, status_forcelist=[429, 413, 503, 401], allowed_methods=["HEAD","GET", "PUT", "DELETE", "OPTIONS", "TRACE"])
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount('https://', adapter)
    
    auth = HTTPBasicAuth('nservice01', 'Testingacc!1')
    
    r = session.get(urlnumjson, headers = headersnum, auth = auth) #headers=headers,
    
    if headersnum == r.request.headers:
        print('True')
    else:
        print('False')
    
    print(f'\n{r.headers}\n', f'\n{r.request.headers}\n')
    
    rnumjson = r.json()


    dfnum = pd.DataFrame(rnumjson)
    recnum = len(dfnum.index)
    
    dfnum = dfnum.sort_values(by=['erFacility','timeResolution','ageCDCILI'], ascending = True).reset_index(drop = True)
    
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Central Dakota Clinic/Charles Nyhus"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Valley City University Student Health"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="UND Center for Family Medicine"
    dfnum.loc[dfnum["erFacility"]=="ND-Essentia Health Fargo Hospital","erFacility"]="Essentia ED"
    dfnum.loc[dfnum["erFacility"]=="ND-Altru Hospital","erFacility"]="Altru ED"
    dfnum.loc[dfnum["erFacility"]=="ND-Sanford Emergency Center Bismarck","erFacility"]="Sanford Medical Center Bismarck"
    dfnum.loc[dfnum["erFacility"]=="ND-Sanford Medical Center Fargo","erFacility"]="Sanford Medical Center Fargo"
    dfnum.loc[dfnum["erFacility"]=="ND-CHI St. Joseph's Health","erFacility"]="St. Alexius Health Dickinson (St. Joseph’s) ED"
    dfnum.loc[dfnum["erFacility"]=="ND-Trinity Hospital","erFacility"]="Trinity ED"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Urgent Medicine Associated/Kurt Kooyer"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="7-Day Clinic/Napoleon Espejo"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Midgarden Family Clinic"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Standing Rock IHS"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Fairlight Medical Center"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Wishek Community Clinic"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Northland Community Health Center McClusky"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="West River Health Services"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Altru Devil’s Lake Family Practice/Lim Elson"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Sanford Walk-in Fargo/Greg Pfennig"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Northland Community Health Center Turtle Lake"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Towner County Medical Center"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="UND Student Health Center"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Trinity Community Clinic New Town"
    dfnum.loc[dfnum["erFacility"]=="","erFacility"]="Trinity Community Clinic Kenmare"
    dfnum.loc[dfnum["erFacility"]=="ND-CHI St. Alexius Williston","erFacility"]="St Alexius Health Williston (Mercy) ED"
    dfnum.loc[dfnum["erFacility"]=="ND-CHI Devils Lake Hospital","erFacility"]="St Alexius Health Devil’s Lake (Mercy) ED"
    dfnum.loc[dfnum["erFacility"]=="ND-Jamestown Regional Medical Center","erFacility"]="Jamestown Regional Medical Center"
    
    periodnum = dfnum['timeResolution'].unique()
    periodnum = periodnum.tolist()
    
    numvarlist = []
    
    for week in enumerate(periodnum):
        
        dfnumweek = dfnum.loc[dfnum['timeResolution'] == week[1]]
    
        dfnumweek = pd.merge(dfnumweek, dfLE, how = 'left', on = ['erFacility'])
        dfnumweek = dfnumweek.drop(columns = ['geographystate'])
        
        weeknum = week[1]
        year = datetime.strptime(str(weeknum) + '-1', '%Y-%W-%w').strftime('%y')
        weeknum = str(week[1])
        weeknum = weeknum.replace('2022-','').replace('2023-','')
        
        dfnumweek['MMWR'] = f'{year}'+f'{weeknum}'
        
        dfnumweek['From'] = 'ILINum'
        
        col = dfnumweek.pop("LE_ID")
        dfnumweek.insert(0, 'Provider ID', col)
        
        col = dfnumweek.pop("MMWR")
        dfnumweek.insert(1, col.name, col)
        
        dfnumweek['ageCDCILI'] = dfnumweek['ageCDCILI'].astype(object)

        #weeknum = str(week[1])
        #weeknum = weeknum.replace('2022-','')
        
        print(weeknum)
        
        numvarlist.append('dfnumweek' + weeknum)
        
        globals()['dfnumweek%s' % weeknum] = dfnumweek
        
        #display(dfnumweek)
        
    numvarlist = [item.strip("'") for item in numvarlist]
        
    print(f'\n{numvarlist}\n')
        
    urldenjson = f'https://essence.syndromicsurveillance.org/nssp_essence/api/tableBuilder?percentParam=noPercent&patientClass=e&medicalGroupingSystem=essencesyndromes&userId=647&geographySystem=state&geography=nd&datasource=va_er&timeResolution=weekly&erFacility=6326&erFacility=6330&erFacility=6360&erFacility=6365&erFacility=6358&erFacility=6320&erFacility=6354&erFacility=6347&erFacility=6369&aqtTarget=TableBuilder&detector=nodetectordetector&fieldIDs=timeResolution&fieldIDs=ageCDCILI&fieldIDs=erFacility&fieldIDs=geographystate&fieldLabels=Week&fieldLabels=CDC%20ILI%20Reporting%20Age%20Group&fieldLabels=Facility&fieldLabels=State&displayTotals=false&displayTotals=false&displayTotals=false&displayTotals=false&displayZeroCountRows=true&graphWidth=431&portletId=285298&dateconfig=16&startDate={startdate}&endDate={enddate}&rowFields=erFacility&rowFields=ageCDCILI&rowFields=timeResolution&columnField=geographystate'
    
    with open(r'C:\Users\gmoye\headerconfigden.json') as g:
        headersden = json.load(g)
    
    r = session.get(urldenjson, headers=headersden, auth = auth) #headers=headers,
    rdenjson = r.json()
    
    if headersden == r.request.headers:
        print('True')
    else:
        print('False')
    
    dfden = pd.DataFrame(rdenjson)
    recden = len(dfden.index)
    
    dfden = dfden.sort_values(by=['erFacility','timeResolution','ageCDCILI'], ascending = True).reset_index(drop = True)
    
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Central Dakota Clinic/Charles Nyhus"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Valley City University Student Health"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="UND Center for Family Medicine"
    dfden.loc[dfden["erFacility"]=="ND-Essentia Health Fargo Hospital","erFacility"]="Essentia ED"
    dfden.loc[dfden["erFacility"]=="ND-Altru Hospital","erFacility"]="Altru ED"
    dfden.loc[dfden["erFacility"]=="ND-Sanford Emergency Center Bismarck","erFacility"]="Sanford Medical Center Bismarck"
    dfden.loc[dfden["erFacility"]=="ND-Sanford Medical Center Fargo","erFacility"]="Sanford Medical Center Fargo"
    dfden.loc[dfden["erFacility"]=="ND-CHI St. Joseph's Health","erFacility"]="St. Alexius Health Dickinson (St. Joseph’s) ED"
    dfden.loc[dfden["erFacility"]=="ND-Trinity Hospital","erFacility"]="Trinity ED"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Urgent Medicine Associated/Kurt Kooyer"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="7-Day Clinic/Napoleon Espejo"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Midgarden Family Clinic"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Standing Rock IHS"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Fairlight Medical Center"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Wishek Community Clinic"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Northland Community Health Center McClusky"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="West River Health Services"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Altru Devil’s Lake Family Practice/Lim Elson"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Sanford Walk-in Fargo/Greg Pfennig"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Northland Community Health Center Turtle Lake"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Towner County Medical Center"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="UND Student Health Center"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Trinity Community Clinic New Town"
    dfden.loc[dfden["erFacility"]=="","erFacility"]="Trinity Community Clinic Kenmare"
    dfden.loc[dfden["erFacility"]=="ND-CHI St. Alexius Williston","erFacility"]="St Alexius Health Williston (Mercy) ED"
    dfden.loc[dfden["erFacility"]=="ND-CHI Devils Lake Hospital","erFacility"]="St Alexius Health Devil’s Lake (Mercy) ED"
    dfden.loc[dfden["erFacility"]=="ND-Jamestown Regional Medical Center","erFacility"]="Jamestown Regional Medical Center"
    
    periodden = dfden['timeResolution'].unique()
    periodden = periodden.tolist()
    
    denvarlist = []
    
    for week in enumerate(periodden):
        
        dfdenweek = dfden.loc[dfden['timeResolution'] == week[1]]
    
        dfdenweek = pd.merge(dfdenweek, dfLE, how = 'left', on = ['erFacility'])
        dfdenweek = dfdenweek.drop(columns = ['geographystate'])
        
        weekden = week[1]
        year = datetime.strptime(str(weekden) + '-1', '%Y-%W-%w').strftime('%y')
        weekden = str(week[1])
        weekden = weekden.replace('2022-','').replace('2023-','')
        
        dfdenweek['MMWR'] = f'{year}'+f'{weekden}'
        
        dfdenweek['From'] = 'ILIDen'
        
        col = dfdenweek.pop("LE_ID")
        dfdenweek.insert(0, 'Provider ID', col)
        
        col = dfdenweek.pop("MMWR")
        dfdenweek.insert(1, col.name, col)
        
        dfdenweek['ageCDCILI'] = dfdenweek['ageCDCILI'].astype(object)
        
        #weekden = str(week[1])
        #weekden = weekden.replace('2022-','')
        
        print(weekden)
        
        denvarlist.append('dfdenweek' + weekden)
        
        globals()['dfdenweek%s' % weekden] = dfdenweek
        
        #display(dfdenweek)
        
    denvarlist = [item.strip("'") for item in denvarlist]
        
    print(f'\n{denvarlist}\n')
    
    periodlist = []
    periodnum.extend(periodden)
    periodlist.extend(periodnum)
    periodlist = [item.replace('2022-','') for item in periodlist]
    periodlist = [item.replace('2023-','') for item in periodlist]
    #periodlist = [item.replace(item,float(item)) for item in periodlist]
    #periodlist = [item.strip("2022") for item in periodlist]
    periodlist = list(set(periodlist))
    print(periodlist, 'periodlist')
    
    periodnum = dfnum['timeResolution'].unique()
    periodnum = periodnum.tolist()
    
    periodnum = [item.strip("2022-") for item in periodnum]
    periodnum = [item.strip("2023-") for item in periodnum]
        
    periodden = dfden['timeResolution'].unique()
    periodden = periodden.tolist()
    
    periodden = [item.strip("2022-") for item in periodden]
    periodden = [item.strip("2023-") for item in periodden]
    
    for value in periodlist:
        
        if any(value in s for s in numvarlist) and any(value in s for s in denvarlist):
            
            print(value)
            
            df1 = 'dfnumweek'
            df2 = 'dfdenweek'
            print(df1+value,df2+value)
            df1 = eval(df1+value)
            df2 = eval(df2+value)
        
            dfsol = pd.merge(df1, df2, how = 'outer')
               
            #dfsol = pd.merge(eval(dfnumweek+value), eval(dfdenweek+value), how = 'outer')
            dfsol = dfsol.sort_values(by=['erFacility','timeResolution','ageCDCILI'], ascending = True).reset_index(drop = True)
            
            dfsol['record_no'] = np.arange(len(dfsol))+1
            rcount = dfsol.pop('record_no')
            dfsol.insert(0, rcount.name, rcount)
            
            dfpivot = dfsol.pivot(index = ['Provider ID','MMWR'], columns = ['ageCDCILI','timeResolution','From'], values=['count'])
            
            df_test = dfsol.pivot(index = ['Provider ID','MMWR'], columns = ['ageCDCILI','timeResolution','From'], values=['count'])
            df_test.columns = df_test.columns.droplevel()
            df_test.columns = [' '.join(col) for col in df_test.columns.values]
            df_test = df_test.reset_index()
            df_test = df_test.set_index(['Provider ID','MMWR'])
            
            df_test['Total patient visits'] = df_test.filter(like="ILIDen").sum(axis=1)
            df_test['total_unk'] = df_test.filter(like="Unknown").sum(axis=1)
            df_test.drop(df_test.columns[[10,11]], axis=1, inplace=True)
            
            display(df_test)
            
            date = datetime.today().strftime('%d%b%y_%H.%M.%S')
            output = fr'Y:\PROGRAM\IMMUNE\Immunize\Influenza\Inf22-23\Surveillance\ESSENCE ILI\Processed\ILI-ND-WEEK{value}-PROCESSED_{date}.xlsx'
            
            df_test.to_excel(f'{output}')
        
        elif any(value in s for s in numvarlist) and any(value not in s for s in denvarlist):
            print(value)
            dfsol = 'dfnumweek'
            print(dfsol+value)
            dfsol = eval(dfsol+value)

               
            #dfsol = pd.merge(eval(dfnumweek+value), eval(dfdenweek+value), how = 'outer')
            dfsol = dfsol.sort_values(by=['erFacility','timeResolution','ageCDCILI'], ascending = True).reset_index(drop = True)
            
            dfsol['record_no'] = np.arange(len(dfsol))+1
            rcount = dfsol.pop('record_no')
            dfsol.insert(0, rcount.name, rcount)
            
            dfpivot = dfsol.pivot(index = ['Provider ID','MMWR'], columns = ['ageCDCILI','timeResolution','From'], values=['count'])
            
            df_test = dfsol.pivot(index = ['Provider ID','MMWR'], columns = ['ageCDCILI','timeResolution','From'], values=['count'])
            df_test.columns = df_test.columns.droplevel()
            df_test.columns = [' '.join(col) for col in df_test.columns.values]
            df_test = df_test.reset_index()
            df_test = df_test.set_index(['Provider ID','MMWR'])
            
            df_test['Total patient visits'] = df_test.filter(like="ILIDen").sum(axis=1)
            df_test['total_unk'] = df_test.filter(like="Unknown").sum(axis=1)
            
            
            display(df_test)
            
        elif any(value not in s for s in numvarlist) and any(value in s for s in denvarlist):
            print(value)
            dfsol = 'dfdenweek'
            print(dfsol+value)
            dfsol = eval(dfsol+value)
            
            #dfsol = pd.merge(eval(dfnumweek+value), eval(dfdenweek+value), how = 'outer')
            dfsol = dfsol.sort_values(by=['erFacility','timeResolution','ageCDCILI'], ascending = True).reset_index(drop = True)
            
            dfsol['record_no'] = np.arange(len(dfsol))+1
            rcount = dfsol.pop('record_no')
            dfsol.insert(0, rcount.name, rcount)
            
            dfpivot = dfsol.pivot(index = ['Provider ID','MMWR'], columns = ['ageCDCILI','timeResolution','From'], values=['count'])
            
            df_test = dfsol.pivot(index = ['Provider ID','MMWR'], columns = ['ageCDCILI','timeResolution','From'], values=['count'])
            df_test.columns = df_test.columns.droplevel()
            df_test.columns = [' '.join(col) for col in df_test.columns.values]
            df_test = df_test.reset_index()
            df_test = df_test.set_index(['Provider ID','MMWR'])
            
            df_test['Total patient visits'] = df_test.filter(like="ILIDen").sum(axis=1)
            df_test['total_unk'] = df_test.filter(like="Unknown").sum(axis=1)
            
            display(df_test)
            
    logoutsecurl = 'https://essence.syndromicsurveillance.org/nssp_essence/j_spring_security_logout'
    logouturl = 'https://essence.syndromicsurveillance.org/nssp_essence/servlet/Login?logoff=true'
        
    logoutsecgetresult = session.get(logoutsecurl, headers = agent, auth = auth)
        
    print(logoutsecgetresult.ok, logoutsecgetresult.status_code)
        
    logoutgetresult = session.get(logouturl, headers = agent)
        
    print(logoutgetresult.ok, logoutgetresult.status_code, dict(referer=logoutsecurl))
    

In [2]:
getData()
transformation()


{'Cache-Control': 'no-cache', 'Set-Cookie': 'JSESSIONID=C53BD29481444FC19649D9605C0E8FB7; Path=/; Secure; HttpOnly', 'Pragma': 'no-cache', 'Content-Type': 'text/html;charset=ISO-8859-1', 'Content-Length': '2156', 'Date': 'Wed, 01 Feb 2023 16:38:30 GMT'}
 
{'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.52', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

{'JSESSIONID': 'C53BD29481444FC19649D9605C0E8FB7'}
JSESSIONID=C53BD29481444FC19649D9605C0E8FB7; Path=/; Secure; HttpOnly
 JSESSIONID=C53BD29481444FC19649D9605C0E8FB7 


{'Strict-Transport-Security': 'max-age=0', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-store', 'Pragma': 'no-cache', 'Expires': 'Wed, 31 Dec 1969 23:59:59 GMT', 'Content-Type': 'text/html;charset=ISO-8859-1', 'Content-Length': '5981', 'Date': 'Wed, 01 Feb 

False
03
04
05

['dfdenweek03', 'dfdenweek04', 'dfdenweek05']

['04', '05', '03'] periodlist
04
dfnumweek04 dfdenweek04


,,00-04 2023-04 ILINum,00-04 2023-04 ILIDen,05-24 2023-04 ILINum,05-24 2023-04 ILIDen,25-49 2023-04 ILINum,25-49 2023-04 ILIDen,50-64 2023-04 ILINum,50-64 2023-04 ILIDen,65+ 2023-04 ILINum,65+ 2023-04 ILIDen,Total patient visits,total_unk
Provider ID,MMWR,,,,,,,,,,,,
38047,2304,0.0,21.0,4.0,99.0,1.0,136.0,0.0,53.0,0.0,60.0,369.0,0.0
38048,2304,4.0,24.0,0.0,62.0,0.0,134.0,2.0,56.0,0.0,51.0,327.0,0.0
38049,2304,5.0,32.0,9.0,90.0,6.0,181.0,2.0,79.0,0.0,60.0,442.0,0.0
38050,2304,1.0,44.0,4.0,140.0,11.0,265.0,3.0,87.0,1.0,59.0,595.0,0.0
38055,2304,2.0,16.0,3.0,38.0,0.0,82.0,0.0,27.0,0.0,38.0,201.0,0.0
38056,2304,5.0,22.0,6.0,100.0,6.0,199.0,3.0,81.0,7.0,120.0,522.0,0.0
38087,2304,2.0,27.0,2.0,58.0,0.0,125.0,0.0,37.0,0.0,34.0,281.0,0.0
38089,2304,3.0,13.0,2.0,32.0,1.0,60.0,2.0,22.0,2.0,18.0,145.0,0.0
38090,2304,3.0,21.0,10.0,43.0,5.0,52.0,5.0,43.0,5.0,47.0,206.0,0.0


05
dfnumweek05 dfdenweek05


,,00-04 2023-05 ILINum,00-04 2023-05 ILIDen,05-24 2023-05 ILINum,05-24 2023-05 ILIDen,25-49 2023-05 ILINum,25-49 2023-05 ILIDen,50-64 2023-05 ILINum,50-64 2023-05 ILIDen,65+ 2023-05 ILINum,65+ 2023-05 ILIDen,Total patient visits,total_unk
Provider ID,MMWR,,,,,,,,,,,,
38047,2305,2.0,11.0,1.0,51.0,1.0,78.0,0.0,17.0,1.0,35.0,192.0,0.0
38048,2305,1.0,16.0,1.0,30.0,3.0,92.0,1.0,27.0,0.0,24.0,189.0,0.0
38049,2305,1.0,11.0,4.0,44.0,2.0,88.0,2.0,32.0,1.0,24.0,199.0,0.0
38050,2305,3.0,25.0,3.0,66.0,5.0,126.0,3.0,45.0,1.0,34.0,296.0,0.0
38055,2305,0.0,4.0,0.0,24.0,0.0,37.0,0.0,9.0,0.0,20.0,94.0,0.0
38056,2305,2.0,10.0,5.0,33.0,2.0,49.0,0.0,28.0,1.0,29.0,149.0,0.0
38087,2305,0.0,6.0,1.0,28.0,1.0,51.0,0.0,16.0,0.0,10.0,111.0,0.0
38089,2305,0.0,1.0,0.0,10.0,1.0,28.0,0.0,12.0,1.0,10.0,61.0,0.0
38090,2305,1.0,3.0,4.0,17.0,2.0,22.0,0.0,13.0,0.0,29.0,84.0,0.0


03
dfnumweek03 dfdenweek03


,,00-04 2023-03 ILINum,00-04 2023-03 ILIDen,05-24 2023-03 ILINum,05-24 2023-03 ILIDen,25-49 2023-03 ILINum,25-49 2023-03 ILIDen,50-64 2023-03 ILINum,50-64 2023-03 ILIDen,65+ 2023-03 ILINum,65+ 2023-03 ILIDen,Total patient visits,total_unk
Provider ID,MMWR,,,,,,,,,,,,
38047,2303,1.0,16.0,0.0,42.0,1.0,80.0,0.0,32.0,0.0,38.0,208.0,0.0
38048,2303,1.0,13.0,4.0,35.0,0.0,83.0,2.0,22.0,1.0,33.0,186.0,0.0
38049,2303,4.0,14.0,5.0,58.0,5.0,88.0,3.0,42.0,2.0,34.0,236.0,0.0
38050,2303,0.0,23.0,2.0,85.0,3.0,137.0,1.0,58.0,1.0,40.0,343.0,0.0
38055,2303,2.0,7.0,1.0,22.0,0.0,51.0,0.0,15.0,1.0,29.0,124.0,0.0
38056,2303,1.0,21.0,4.0,63.0,4.0,95.0,1.0,53.0,4.0,63.0,295.0,0.0
38087,2303,2.0,23.0,6.0,40.0,2.0,64.0,0.0,26.0,1.0,19.0,172.0,0.0
38089,2303,1.0,5.0,2.0,31.0,1.0,36.0,1.0,16.0,0.0,16.0,104.0,0.0
38090,2303,0.0,12.0,4.0,18.0,1.0,31.0,0.0,19.0,0.0,28.0,108.0,0.0


True 200
True 200 {'referer': 'https://essence.syndromicsurveillance.org/nssp_essence/j_spring_security_logout'}


In [ ]:
import sys
import warnings
import pandas as pd
import numpy as np
import os
import cx_Oracle as O
import datetime as datetime
import logging, logging.handlers, smtplib
import traceback
import smtplib
import sqlalchemy as sa
import pyodbc
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from os.path import basename
import os
from datetime import datetime, date, timedelta
import settings
import errhandler as eh
import eHSQL
import re
import os
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from requests.auth import HTTPBasicAuth
import json
from lxml import html
import string
import bs4
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

global r, rnumjson

filepath = r'Processed'
try:
    
    logging.basicConfig(filename = 'ILIWebAutomation.log',
                       filemode='a',
                       format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    def check_files():
        global processlist
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def convert_to_CSV(doc):
        #global df
        pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name='Daily data report').to_csv(f'{filepath}'+'/'+'DataAndDuplicateReport' + '_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.csv', encoding = 'utf-8', index = False)
        shutil.move(f'{filepath}'+'/'+f'{doc}', 'DataAndDuplicateReportAutomation/Archive/xlsx/'+f'{doc}')
        #for f in glob('DataAndDuplicateReportAutomation/DataAndDuplicateReport*.csv'):
            #df = pd.read_csv(f)
        #display(df)
        return #df
    
    def process_CSV():
        global processlist, df
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport") and file.endswith(".csv"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def move_processed():
        global processedlist
        processedlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DDRProcessed") and file.endswith(".csv"):
                print(f'\n{file}')
                processedlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DDRProcessed"):
                print('incorrect file name is present.')
        if processedlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processedlist += ['None']
        else:
            pass
        return processedlist

except Exception as e:
    exc_type, exc_obj, exc_tb = sys.exc_info()
    fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
    print(e, 'line: ', exc_tb.tb_lineno,'\n\n', traceback.format_exc())
    pass

except (Exception, O.Error) as dberr:
    print(dberr)
    logging.exception("message")
    pass

In [ ]:
import glob
from glob import glob
import shutil
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading
import os

global processlist, df
conn = False
con = None

filepath = fr'{print(os.path.basename(os.getcwd()))}'
try:
    
    logging.basicConfig(filename = 'RepliconCSVConversion.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    def check_folders():
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith(("Archive")):
                pass
            elif ~file.startswith("Archive"):
                os.path.exists(filepath, r'/Archive')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist

    def check_files():
        global processlist
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def convert_to_CSV(doc):
        #global df
        pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name='Daily data report').to_csv(f'{filepath}'+'/'+'DataAndDuplicateReport' + '_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.csv', encoding = 'utf-8', index = False)
        shutil.move(f'{filepath}'+'/'+f'{doc}', 'DataAndDuplicateReportAutomation/Archive/xlsx/'+f'{doc}')
        #for f in glob('DataAndDuplicateReportAutomation/DataAndDuplicateReport*.csv'):
            #df = pd.read_csv(f)
        #display(df)
        return #df
    
    check_folders()
    

    
#exceptions
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    print("All errors identified.")
    if conn == True:
        close_connection()
        cur.close()


In [ ]:
import glob
from glob import glob
import shutil
import requests
import urllib
import itertools
from requests.exceptions import ConnectionError, HTTPError, Timeout, TooManyRedirects
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import traceback
import logging
import logging.handlers
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy as sa
from sqlalchemy import create_engine, event
import json
import time as ti
import datetime
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import re
import os
import asyncio
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import ctypes
import threading
import os

global processlist, df
conn = False
con = None

filepath = fr'{print(os.path.basename(os.getcwd()))}'
try:
    
    logging.basicConfig(filename = 'RepliconCSVConversion.log',
                        filemode='a',
                        format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                        datefmt='%H:%M:%S',
                        level=logging.DEBUG)
    logging.info(f'\nLOG START: {datetime.now()}\n')
    
    def check_folders():
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith(("Archive")):
                pass
            elif ~file.startswith("Archive"):
                os.path.exists(filepath, r'/Archive')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist

    def check_files():
        global processlist
        processlist = []
        for file in os.listdir(filepath):
            print(f'{file}')
            if file.startswith("DataAndDuplicateReport"):
                print(f'\n{file}')
                processlist += [file]
            elif file.startswith(("Archive","Analysis","Temp")):
                pass
            elif ~file.startswith("DataAndDuplicateReport"):
                print('incorrect file name is present.')
        if processlist == []:
            print('No files were present or did not have the correct naming scheme.')
            processlist += ['None']
        else:
            pass
        return processlist
    
    def convert_to_CSV(doc):
        #global df
        pd.read_excel(f'{filepath}'+'/'+f'{doc}', sheet_name='Daily data report').to_csv(f'{filepath}'+'/'+'DataAndDuplicateReport' + '_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.csv', encoding = 'utf-8', index = False)
        shutil.move(f'{filepath}'+'/'+f'{doc}', 'DataAndDuplicateReportAutomation/Archive/xlsx/'+f'{doc}')
        #for f in glob('DataAndDuplicateReportAutomation/DataAndDuplicateReport*.csv'):
            #df = pd.read_csv(f)
        #display(df)
        return #df
    
    check_folders()
    
    #check_files()
    #if 'None' not in processlist:
    #    print('\ndocuments available for processing\n')
    #    print(processlist)
    #    for doc in enumerate(processlist):
    #        try:
    #            gstart_time = datetime.now()
    #            convert_to_CSV(doc[1])
    #            gend_time = datetime.now()
    #            print("")
    #            print('\nDuration: {}'.format(gend_time - gstart_time))
    #            logging.info('\nDuration: {}'.format(gend_time - gstart_time))
    #        except Exception as e:
    #            print("")
    #            print(e)
    #            logging.exception('\n')
    #            logging.exception("message")
    #        for i in range(10, -1, -1):
    #                print(f"{i} seconds until next document is processed", end = "\r")
    #                ti.sleep(1)
    #else:
    #    print('documents not available for processing')
    #    pass
    #
    #if 'None' not in processedlist:
    #    for file in processedlist:
    #        shutil.move(f'{filepath}'+'/'+f'{processedlist[0]}', 'DataAndDuplicateReportAutomation/Archive/csv/'+f'{processedlist[0]}')
    #        move_processed()
    #    print('Processing Complete')
    #else:
    #    pass
    
#exceptions 
except (Exception, pyodbc.DatabaseError) as error:
        print(error)
        logging.exception("message")
        pass
    
except requests.exceptions.HTTPError as errh:
    print("Http Error:",  errh)
    conn = False
    logging.exception("message")
    
except requests.exceptions.ConnectionError as errc:
    print("Error Connecting:", errc)
    conn = False
    logging.exception("message")
    
except requests.exceptions.Timeout as errt:
    print("Timeout Error:", errt)
    conn = False
    logging.exception("message")
    
except requests.exceptions.RequestException as erru:
    print("Unidentified Request Exception:", erru)
    conn = False
    logging.exception("message")
        
finally:
    logging.info(f'\nLOG END: {datetime.now()}\n')
    print("All errors identified.")
    if conn == True:
        close_connection()
        cur.close()
